<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Meta/blob/master/tutorials/1-Introduction/FinRL_PortfolioAllocation_NeurIPS_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys

# Option 1:
print(sys.version_info)

# Option 2:
print(sys.version)


sys.version_info(major=3, minor=10, micro=11, releaselevel='final', serial=0)
3.10.11 (main, May 17 2023, 14:30:36) [Clang 14.0.6 ]


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [3]:
# ## install finrl library
# %pip install wrds
# %pip install swig
# %pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [14]:
from config import config



<a id='1.3'></a>
## 2.3. Import Packages

In [13]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
import datetime
from stockstats import StockDataFrame as Sdf
from config import config
import config_tickers
from yahoodownloader import YahooDownloader
from preprocessors import FeatureEngineer, data_split
from env_portfolio import StockPortfolioEnv
from models.models import DRLAgent
from plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
from data_processor import DataProcessor
from processor_yahoofinance import YahooFinanceProcessor
import sys
sys.path.append("../FinRL-Library")

ImportError: cannot import name 'FeatureEngineer' from 'preprocessors' (unknown location)

<a id='1.4'></a>
## 2.4. Create Folders

In [5]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [6]:
full_start_date="2004-01-01"
full_end_date="2023-07-19"
train_start_date="2004-01-01"
train_end_date="2015-12-31"
validate_start_date="2011-01-01"
validate_end_date="2015-12-31"
test_start_date="2016-01-01"
test_end_date="2023-07-19"
etf_list =["SPY","QQQ","IWM","EEM"]
flo_ticker = ["AAPL","MSFT", "NVDA", "INTC","GOOGL","TSLA","DAL"]

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

<a id='4'></a>
# Flo - Add own Indicators
TBD

In [7]:
# df.to_csv("etf.csv")

In [8]:
test = df.date.factorize()[0]
test

NameError: name 'df' is not defined

In [ ]:
df.index = df.date.factorize()[0]
lookrange=252
for i in range(lookrange,len(df.index.unique())):
  data_lookback = df.loc[i-lookrange:i,:]
  print(data_lookback)


## Add covariance matrix as states

In [ ]:
# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values 
  cov_list.append(covs)

  
df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)
        

In [ ]:
# df.to_csv("testcov.csv")

In [ ]:
df
# df.to_csv("etf_for_optuna.csv")

<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.


## Training data split: 2009-01-01 to 2020-07-01

In [ ]:
train = data_split(df, train_start_date, train_end_date)
validate = data_split(df, validate_start_date, validate_end_date)
#trade = data_split(df, '2020-01-01', config.END_DATE)

In [ ]:
# train.to_csv("train.csv")

In [ ]:
train

## Environment for Portfolio Allocation


In [ ]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,)) 
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)   
        self.terminal = False       
        self.turbulence_threshold = turbulence_threshold        
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()
            
            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))           
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions) 
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            #print(self.state)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])            
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value 
            #print("Step reward: ", self.reward)
            #self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [9]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


NameError: name 'train' is not defined

In [ ]:
# new_indicators = ["macds","macdh","rsi_6","rsi_14","close_200_sma","stochrsi","stochrsi_6","trix","vr","wr","cci","atr","supertrend","supertrend_ub","supertrend_lb","dma","pdi"]
all_indics = [
        "macd",
        "boll_ub",
        "boll_lb",
        "rsi_30",
        "cci_30",
        "dx_30",
        "close_30_sma",
        "close_60_sma",
        "macds","macdh","rsi_6","rsi_14","close_200_sma","stochrsi","stochrsi_6","trix","vr","wr","cci","atr","supertrend","supertrend_ub","supertrend_lb","dma","pdi"
    ]

env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    # "tech_indicator_list": config.INDICATORS, 
    "tech_indicator_list": all_indics, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)
#!FLO New validation set
e_validate_gym = StockPortfolioEnv(df = validate, **env_kwargs)

In [ ]:
env_train, _ = e_train_gym.get_sb_env()
#!FLO New validation set
env_validate, _ = e_validate_gym.get_sb_env()
print(type(env_train))

HYPEROPT

<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [ ]:
# initialize
agent = DRLAgent(env = env_train)


### Model 1: **A2C**


In [ ]:
#!My Best Values
best_learning_rate= 3.3798011562953303e-05
best_n_steps= 37
best_gamma= 0.9912299824042836
best_gae_lambda= 0.922570232166817
best_ent_coef= 0.07308895723592097
best_vf_coef= 0.24495410191752645
best_max_grad_norm= 0.4750542295391215

agent = DRLAgent(env = env_train)
A2C_PARAMS = {"n_steps": best_n_steps, "ent_coef": best_ent_coef, "learning_rate": best_learning_rate, "gamma":best_gamma, "gae_lambda":best_gae_lambda, "vf_coef":best_vf_coef, "max_grad_norm":best_max_grad_norm}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS, tensorboard_log='/Users/floriankockler/Documents/GitHub.nosync/TrainedModels/TensorboardLogs/')
#OLD

# agent = DRLAgent(env = env_train)

# A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
# model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS, tensorboard_log='/Users/floriankockler/Documents/GitHub.nosync/TrainedModels/TensorboardLogs/')

In [ ]:
trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c_etf_best_params',
                                total_timesteps=50000)

    

In [ ]:
trained_a2c.save('/Users/floriankockler/Documents/GitHub.nosync/TrainedModels//trained_a2c_etf_new2.zip')

### Model 2: **PPO**


In [ ]:
# agent = DRLAgent(env = env_train)
# PPO_PARAMS = {
#     "n_steps": 2048,
#     "ent_coef": 0.005,
#     "learning_rate": 0.0001,
#     "batch_size": 128,
# }
# model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

In [ ]:
# trained_ppo = agent.train_model(model=model_ppo, 
#                              tb_log_name='ppo',
#                              total_timesteps=80000)

In [ ]:
# trained_ppo.save('/Users/floriankockler/Documents/GitHub.nosynctrained_ppo_etf_us.zip')

### Model 3: **DDPG**


In [ ]:
# agent = DRLAgent(env = env_train)
# DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}


# model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

In [ ]:
# trained_ddpg = agent.train_model(model=model_ddpg, 
#                              tb_log_name='ddpg',
#                              total_timesteps=50000)

In [ ]:
# trained_ddpg.save('/content/trained_models/trained_ddpg.zip')

### Model 4: **SAC**


In [ ]:
# agent = DRLAgent(env = env_train)
# SAC_PARAMS = {
#     "batch_size": 128,
#     "buffer_size": 100000,
#     "learning_rate": 0.0003,
#     "learning_starts": 100,
#     "ent_coef": "auto_0.1",
# }

# model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

In [ ]:
# trained_sac = agent.train_model(model=model_sac, 
#                              tb_log_name='sac',
#                              total_timesteps=50000)

In [ ]:
# trained_sac.save('/content/trained_models/trained_sac.zip')

### Model 5: **TD3**


In [ ]:
# agent = DRLAgent(env = env_train)
# TD3_PARAMS = {"batch_size": 100, 
#               "buffer_size": 1000000, 
#               "learning_rate": 0.001}

# model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

In [ ]:
# trained_td3 = agent.train_model(model=model_td3, 
#                              tb_log_name='td3',
#                              total_timesteps=30000)

In [ ]:
# trained_td3.save('/content/trained_models/trained_td3.zip')

HYPEROPT

## Trading
Assume that we have $1,000,000 initial capital at 2019-01-01. We use the A2C model to trade Dow jones 30 stocks.

In [ ]:
trade = data_split(df,test_start_date, test_end_date)
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)



In [ ]:
trade.shape
trade

In [ ]:
# df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_a2c,
#                         environment = e_trade_gym)

#my own
df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)


In [ ]:
df_daily_return.head()

In [ ]:
df_daily_return.to_csv('df_daily_return.csv')

In [ ]:
df_actions.head()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))

# assuming actions_df is your DataFrame
ax.stackplot(df_actions.index, df_actions.T, labels=df_actions.columns)

ax.set_title('Portfolio Allocation Over Time')
ax.set_xlabel('Date')
ax.set_ylabel('Allocation')
ax.legend(loc='upper left')

plt.show()

In [ ]:
df_actions.to_csv('df_actions.csv')

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
from pyfolio import timeseries
DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
perf_func = timeseries.perf_stats 
perf_stats_all = perf_func( returns=DRL_strat, 
                              factor_returns=DRL_strat, 
                                positions=None, transactions=None, turnover_denom="AGB")

In [ ]:
# df_daily_return["date"] = pd.to_datetime(df_daily_return["date"])
df_daily_return

In [ ]:
print("==============DRL Strategy Stats===========")
perf_stats_all

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="QQQ", 
        start = df_daily_return.loc[0,'date'],
        end = df_daily_return.loc[len(df_daily_return)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
import pyfolio
%matplotlib inline

baseline_df = get_baseline(
        ticker='SPY', start=df_daily_return.loc[0,'date'], end=test_end_date
    )

baseline_returns = get_daily_return(baseline_df, value_col_name="close")

with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(returns = DRL_strat,
        
                                       benchmark_rets=baseline_returns, set_context=False)



## Min-Variance Portfolio Allocation

In [ ]:
# %pip install PyPortfolioOpt

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models

In [ ]:
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

In [ ]:
unique_trade_date

In [ ]:
df.head()

In [ ]:
#calculate_portfolio_minimum_variance
portfolio = pd.DataFrame(index = range(1), columns = unique_trade_date)
initial_capital = 1000000
portfolio.loc[0,unique_trade_date[0]] = initial_capital

for i in range(len( unique_trade_date)-1):
    df_temp = df[df.date==unique_trade_date[i]].reset_index(drop=True)
    df_temp_next = df[df.date==unique_trade_date[i+1]].reset_index(drop=True)
    #Sigma = risk_models.sample_cov(df_temp.return_list[0])
    #calculate covariance matrix
    Sigma = df_temp.return_list[0].cov()
    #portfolio allocation
    ef_min_var = EfficientFrontier(None, Sigma,weight_bounds=(0, 0.6))
    #minimum variance
    raw_weights_min_var = ef_min_var.min_volatility()
    #get weights
    cleaned_weights_min_var = ef_min_var.clean_weights()
    
    #current capital
    cap = portfolio.iloc[0, i]
    #current cash invested for each stock
    current_cash = [element * cap for element in list(cleaned_weights_min_var.values())]
    # current held shares
    current_shares = list(np.array(current_cash)
                                      / np.array(df_temp.close))
    # next time period price
    next_price = np.array(df_temp_next.close)
    ##next_price * current share to calculate next total account value 
    portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)
    
portfolio=portfolio.T
portfolio.columns = ['account_value']

In [ ]:
portfolio.head()

In [ ]:
a2c_cumpod =(df_daily_return.daily_return+1).cumprod()-1

In [ ]:
min_var_cumpod =(portfolio.account_value.pct_change()+1).cumprod()-1

In [ ]:
dji_cumpod =(baseline_returns+1).cumprod()-1

## Plotly: DRL, Min-Variance, DJIA

In [ ]:
# %pip install plotly

In [ ]:
from datetime import datetime as dt

import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go

In [ ]:
time_ind = pd.Series(df_daily_return.date)

In [ ]:
trace0_portfolio = go.Scatter(x = time_ind, y = a2c_cumpod, mode = 'lines', name = 'Model Portfolio')

trace1_portfolio = go.Scatter(x = time_ind, y = dji_cumpod, mode = 'lines', name = 'DJIA')
trace2_portfolio = go.Scatter(x = time_ind, y = min_var_cumpod, mode = 'lines', name = 'Min-Variance')
#trace3_portfolio = go.Scatter(x = time_ind, y = ddpg_cumpod, mode = 'lines', name = 'DDPG')
#trace4_portfolio = go.Scatter(x = time_ind, y = addpg_cumpod, mode = 'lines', name = 'Adaptive-DDPG')
#trace5_portfolio = go.Scatter(x = time_ind, y = min_cumpod, mode = 'lines', name = 'Min-Variance')

#trace4 = go.Scatter(x = time_ind, y = addpg_cumpod, mode = 'lines', name = 'Adaptive-DDPG')

#trace2 = go.Scatter(x = time_ind, y = portfolio_cost_minv, mode = 'lines', name = 'Min-Variance')
#trace3 = go.Scatter(x = time_ind, y = spx_value, mode = 'lines', name = 'SPX')

In [ ]:
fig = go.Figure()
fig.add_trace(trace0_portfolio)

fig.add_trace(trace1_portfolio)

fig.add_trace(trace2_portfolio)



fig.update_layout(
    legend=dict(
        x=0,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=15,
            color="black"
        ),
        bgcolor="White",
        bordercolor="white",
        borderwidth=2
        
    ),
)
#fig.update_layout(legend_orientation="h")
fig.update_layout(title={
        #'text': "Cumulative Return using FinRL",
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
#with Transaction cost
#fig.update_layout(title =  'Quarterly Trade Date')
fig.update_layout(
#    margin=dict(l=20, r=20, t=20, b=20),

    paper_bgcolor='rgba(1,1,0,0)',
    plot_bgcolor='rgba(1, 1, 0, 0)',
    #xaxis_title="Date",
    yaxis_title="Cumulative Return",
xaxis={'type': 'date', 
       'tick0': time_ind[0], 
        'tickmode': 'linear', 
       'dtick': 86400000.0 *80}

)
fig.update_xaxes(showline=True,linecolor='black',showgrid=True, gridwidth=1, gridcolor='LightSteelBlue',mirror=True)
fig.update_yaxes(showline=True,linecolor='black',showgrid=True, gridwidth=1, gridcolor='LightSteelBlue',mirror=True)
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='LightSteelBlue')

fig.show()

OPTUNA

In [ ]:
import optuna
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy

def objective(trial):
    # Define a search space for hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    n_steps = trial.suggest_int("n_steps", 1, 10)
    ent_coef = trial.suggest_float("ent_coef", 0.001, 0.1, log=True)
    
    # Create the A2C model
    model_kwargs = {"n_steps": n_steps, "ent_coef": ent_coef, "learning_rate": learning_rate}
    model = agent.get_model(model_name="a2c", model_kwargs=model_kwargs, tensorboard_log='/Users/floriankockler/Documents/GitHub.nosync/TrainedModels/TensorboardLogs')

    # Train the model
    model = agent.train_model(model=model, tb_log_name='a2c_etf', total_timesteps=50000)

    # Evaluate the model and get the average reward over 100 steps
    mean_reward, _ = evaluate_policy(model, env_train, n_eval_episodes=100)
    
    return mean_reward  # Optuna maximizes the returned value

# Create a study object and optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# Get the best parameters
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
# def objective(trial):
#     # Define hyperparameters
#     learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
#     n_steps = trial.suggest_int("n_steps", 1, 50)
#     gamma = trial.suggest_float("gamma", 0.9, 0.9999)
#     gae_lambda = trial.suggest_float("gae_lambda", 0.8, 1.0)
#     ent_coef = trial.suggest_float("ent_coef", 1e-6, 0.1, log=True)
#     vf_coef = trial.suggest_float("vf_coef", 0.1, 0.9)

#     model = A2C('MlpPolicy', env_train, learning_rate=learning_rate, 
#                 n_steps=n_steps, gamma=gamma, gae_lambda=gae_lambda, 
#                 ent_coef=ent_coef, vf_coef=vf_coef, verbose=0)

#     model.learn(total_timesteps=50000)
    
#     # Test the trained model
#     test_env = e_trade_gym
#     test_obs = test_env.reset()
#     done = False
#     total_reward = 0

#     while not done:
#         action, _ = model.predict(test_obs)
#         test_obs, reward, done, info = test_env.step(action)
#         total_reward += reward
    
#     return total_reward  # Objective value

# # Optimize
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=100)

# # Display the best trial
# print("Best trial:")
# trial = study.best_trial
# print(" Value: ", trial.value)
# print(" Params: ")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")



In [ ]:
import joblib
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    n_steps = trial.suggest_int("n_steps", 5, 50)
    gamma = trial.suggest_float("gamma", 0.9, 0.9999, log=True)
    gae_lambda = trial.suggest_float("gae_lambda", 0.8, 1.0, log=True)
    ent_coef = trial.suggest_float("ent_coef", 0.001, 0.1, log=True)
    vf_coef = trial.suggest_float("vf_coef", 0.1, 0.5, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.1, 0.6, log=True)

    model = A2C(
        "MlpPolicy",
        env_train,
        learning_rate=learning_rate,
        n_steps=n_steps,
        gamma=gamma,
        gae_lambda=gae_lambda,
        ent_coef=ent_coef,
        vf_coef=vf_coef,
        max_grad_norm=max_grad_norm,
        verbose=0,
    )
    model.learn(total_timesteps=20000)

    _, rewards = evaluate_policy(model, e_trade_gym, n_eval_episodes=5, return_episode_rewards=True)
    return np.mean(rewards)
def save_checkpoint(study, trial):
    joblib.dump(study, 'checkpoint.pkl')
# Optimize
study = optuna.create_study(direction="maximize")

# Check if a checkpoint exists
try:
    study = joblib.load('checkpoint.pkl')
except FileNotFoundError:
    pass

study.optimize(objective, n_trials=100)

# Display the best trial
print("Best trial:")
trial = study.best_trial


print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


Optuna for A2C

In [ ]:
import optuna
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
def objective(trial):
    # hyperparameters
    # gamma = trial.suggest_categorical('gamma', [0.99, 0.995, 0.999, 0.9999])
    # n_steps = trial.suggest_categorical('n_steps', [5, 10, 20, 50, 100])
    # ent_coef = trial.suggest_loguniform('ent_coef', 0.00001, 0.1)
    # learning_rate = trial.suggest_loguniform('learning_rate', 0.00001, 0.1)
    # vf_coef = trial.suggest_loguniform('vf_coef', 0.1, 0.9)
    # max_grad_norm = trial.suggest_categorical('max_grad_norm', [0.3, 0.5, 0.7, 0.9, 1])
    # gae_lambda = trial.suggest_categorical('gae_lambda', [0.9, 0.92, 0.95, 0.98, 1.0])
    #Hier mit floating, dauert länger
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    n_steps = trial.suggest_int("n_steps", 5, 50)
    gamma = trial.suggest_float("gamma", 0.9, 0.9999, log=True)
    gae_lambda = trial.suggest_float("gae_lambda", 0.8, 1.0, log=True)
    ent_coef = trial.suggest_float("ent_coef", 0.001, 0.1, log=True)
    vf_coef = trial.suggest_float("vf_coef", 0.1, 0.5, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.1, 0.6, log=True)
    
    # Create the model
    model = A2C("MlpPolicy", env_train, 
                gamma=gamma, 
                n_steps=n_steps, 
                ent_coef=ent_coef, 
                learning_rate=learning_rate, 
                vf_coef=vf_coef,
                max_grad_norm=max_grad_norm,
                gae_lambda=gae_lambda,
                verbose=0)

    # Here I'm supposing your environment has attributes reward_scaling, lookback and transaction_cost_pct,
    # which you would like to optimize. Replace with the attributes of your own environment.
    env_train.reward_scaling = trial.suggest_uniform("reward_scaling", 0.1, 10)
    env_train.lookback = trial.suggest_int("lookback", 100, 500)


    # Train the model
    model.learn(total_timesteps=50000)

    # Test the model and return the mean reward
    mean_reward, _ = evaluate_policy(model, e_trade_gym, n_eval_episodes=10)
    return mean_reward

# Create a study object and optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print('Best trial:')
trial_ = study.best_trial

print(f'Value: {trial_.value}')

print('Best hyperparameters:')
for key, value in trial_.params.items():
    print(f'    {key}: {value}')
